In [2]:
from bertopic import BERTopic
from scipy.cluster import hierarchy as sch
import pandas as pd
import logging
from sklearn.metrics.pairwise import cosine_distances
from tqdm import tqdm
import numpy as np
from umap import UMAP
from hdbscan import HDBSCAN
from sentence_transformers import SentenceTransformer
import re
import sys
sys.path.append('../../')
from utility.utility_functions import *

In [3]:
#  '\\\\Cm\\CM\\'

'\r\n--------------------------\r\nSent from my BlackBerry Wireless Handheld (www.BlackBerry.net)',


In [ ]:
df = pd.read_parquet('../../data/processed/05_avocado_cleaned_clustered_wip.parquet', engine='pyarrow')

,text
0,Bug 5539\r\nThe screen wizard now ignores the ...
1,"At 02:50 PM 3/15/2001 +0100, it seemed like Ma..."
2,"All,\r\nWe are still in a Nor'easter, but the ..."
3,Responding to Robert Sartin (07:02 AM 6/14/200...
4,I will try to get the real secure up at the fi...
...,...
128640,Checked in WinInetBrowser.java to fix a bug in...
128641,Just wanted to make sure that you know I did t...
128642,"In the end, you will always fail to think of e..."
128643,"Yes, it's valid for Europe, as well. Venk for..."


In [5]:
texts = df['text']

In [9]:
test = texts[texts.str.startswith('\\\\')]
testest = test.tolist()
testest.sort()
testest

['\\\\Cm\\Burn\\4.5GA_Germanium.x\\05-06-2002\\003\\fullinstaller',
 "\\\\Cm\\CM\\Build\\4.0GA_palladium.x\\04-01-2002\\136\\fullinstaller\\setup.exe\r\nWith this build, Full Installer seems working fine, but there are problems.\r\n1. You can not run OCA.  If you try to run OCA, from d:\\build\\bin\\runOCA.bat --> it just doesn't run\r\n2. If you try to write your own batch file to run OCA, since the schema.xml is not there, the Biogen Application won't run\r\n3. The convertutil.jar is not there in the build.  So we can not use Roopak's utility.\r\nSreenivas",
 '\\\\Cm\\CM\\Build\\4.5GA_Germanium.x\\05-07-2002\\004\\fullinstaller',
 '\\\\Cm\\CM\\Build\\4.5GA_Germanium.x\\06-26-2002\\1090\\fullinstaller\\setup.exe\r\nHi Wilhan,\r\nInstaller fails to install the online.  Do you know how to proceed?',
 '\\\\Cm\\CM\\Build\\4.5GA_Germanium.x\\06-27-2002\\037\\fullinstaller',
 '\\\\Cm\\CM\\Build\\4.5GA_Germanium.x\\07-08-2002\\039\\fullinstaller',
 '\\\\Cm\\CM\\Build\\4.5GA_Germanium.x\\07-2

In [8]:
for t in test:
    print(repr(t))

"\\\\Cm\\CM\\Build\\4.0GA_palladium.x\\04-01-2002\\136\\fullinstaller\\setup.exe\r\nWith this build, Full Installer seems working fine, but there are problems.\r\n1. You can not run OCA.  If you try to run OCA, from d:\\build\\bin\\runOCA.bat --> it just doesn't run\r\n2. If you try to write your own batch file to run OCA, since the schema.xml is not there, the Biogen Application won't run\r\n3. The convertutil.jar is not there in the build.  So we can not use Roopak's utility.\r\nSreenivas"
'\\\\Cm\\CM\\Build\\4.5GA_Germanium.x\\10-03-2002\\132\\fullinstaller'
'\\\\Cm\\CM\\Build\\4.6GA_Neon.x\\12-25-2002\\043\\fullinstaller\r\nThe EMDS online tests have passed over this build.\r\nSreenivas'
'\\\\yuyu\\software\\jprobe\r\ncheck sno.txt for license key.\r\n-yuyu'
'\\\\Cm\\CM\\Build\\4.5GA_Germanium.x\\10-14-2002\\140\\fullinstaller'
'\\\\Cm\\CM\\Build\\4.5GA_Germanium.x\\09-30-2002\\129'
'\\\\Utilitysrv1\\home\\Marketing\\solutions_folder'
'\\\\utilitysrv1\\home\\SalesEngineering\\Accou

In [ ]:
with open("signatures.txt", "r") as file:
    signature_list = [line.strip() for line in file]

for signature in signature_list:
    texts = texts.str.replace(signature,"")

In [ ]:
texts = texts[~(   texts.str.contains('Whether you are directly or indirectly')  )]

In [ ]:
pattern = re.compile(pattern=r'\r\n----- Original Message -----\r\n.*',flags=re.DOTALL|re.MULTILINE)
texts = texts.apply(lambda x: re.sub(pattern,"", x))

texts = texts[~texts.str.contains('If you would like to stop or modify the delivery of Research via Email')]
texts = texts[~texts.str.contains('You are invited to join a meeting on the web, using WebEx.')]
texts = texts[~texts.str.contains('REGISTER TODAY')]
texts = texts[~texts.str.contains('-------------------------------------------------------------------------\r\nThis message has been sent to you electronically')]
texts = texts[~texts.str.startswith('~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~')]

# Remove Yahoo
pattern = re.compile(pattern=r'(?:\r\n)?\_+\r\nDo You Yahoo!\?\r\n.*?yahoo\.com.*?\r\n', flags=re.DOTALL | re.MULTILINE )
texts = texts.apply(lambda x: re.sub(pattern,"", x))


pt = r'^Return-Path: <.*?@.*?\.\w{2,3}>\r\n'
pt = re.compile(pt,flags=re.DOTALL|re.MULTILINE | re.IGNORECASE)
texts = texts.apply(lambda x: re.sub(pt,"", x))

pt = re.compile(r'^--=_alternative\s[A-F\d]{16}_=--(?:\r\n)?',flags=re.DOTALL|re.MULTILINE | re.IGNORECASE)
texts = texts.apply(lambda x: re.sub(pt,"", x))

pt = re.compile(r'^--=_related\s[A-F\d]{16}_=.*?[A-F\d]{16}_=--\r\n',flags=re.DOTALL|re.MULTILINE | re.IGNORECASE)
texts = texts.apply(lambda x: re.sub(pt,"", x))

pt = re.compile(r'^-{5}\sOriginal\sMessage\s-{5}(?:\s)?\r\n(?:From:.*?\r\nTo:.*?\r\nSubject:.*?\r\n)?',flags=re.DOTALL|re.MULTILINE | re.IGNORECASE)
texts = texts.apply(lambda x: re.sub(pt,"", x))

pt = re.compile(r'^Date:.*?Message-ID:.*?\r\n',flags=re.DOTALL|re.MULTILINE | re.IGNORECASE)
texts = texts.apply(lambda x: re.sub(pt,"", x))

pt = re.compile(r'^Date:.*?may not be legible.\r\n',flags=re.DOTALL|re.MULTILINE | re.IGNORECASE)
texts = texts.apply(lambda x: re.sub(pt,"", x))

pt = re.compile(r'^Date:.*?iso-8859-1"\r\n',flags=re.DOTALL|re.MULTILINE | re.IGNORECASE)
texts = texts.apply(lambda x: re.sub(pt,"", x))

pt = re.compile(r'^Date:  [\/\d]{10}\s[\d:]{8}\r\n',flags=re.DOTALL|re.MULTILINE | re.IGNORECASE)
texts = texts.apply(lambda x: re.sub(pt,"", x))

pt = re.compile(r'^Date: .*?MIME-Version.*text\/plain\r\n',flags=re.DOTALL|re.MULTILINE | re.IGNORECASE)
texts = texts.apply(lambda x: re.sub(pt,"", x))

pt = re.compile(r'^From: ".+?" <.*?\@.*?\.[a-z]{2,3}>\r\n',flags=re.DOTALL|re.MULTILINE | re.IGNORECASE)
texts = texts.apply(lambda x: re.sub(pt,"", x))

pt = re.compile(r'\s\(www\.avocadoit.com\)$',flags=re.DOTALL|re.MULTILINE | re.IGNORECASE)
texts = texts.apply(lambda x: re.sub(pt,"", x))

0         Bug 5539\r\nThe screen wizard now ignores the ...
1         At 02:50 PM 3/15/2001 +0100, it seemed like Ma...
2         All,\r\nWe are still in a Nor'easter, but the ...
3         Responding to Robert Sartin (07:02 AM 6/14/200...
4         I will try to get the real secure up at the fi...
                                ...                        
128640    Checked in WinInetBrowser.java to fix a bug in...
128641    Just wanted to make sure that you know I did t...
128642    In the end, you will always fail to think of e...
128643    Yes, it's valid for Europe, as well.  Venk for...
128644    Hey John,\r\nThe Siebel Adapter will allow you...
Name: text, Length: 128641, dtype: object

## Save to DF

In [11]:
df = texts.to_frame()
df.to_parquet('../../data/processed/05_avocado_cleaned_clustered_wip_7.parquet', engine='pyarrow')

In [ ]:
len(df)